In [107]:
from pathlib import Path
from osgeo import ogr, gdal
import os
import json
import shutil

gdal.UseExceptions()


In [108]:
current_folder = Path(os.getcwd())
data = json.load(open(current_folder / 'tests' / 'download.json'))

# output_dataset = ogr.GetDriverByName('OpenFileGDB').CreateDataSource(current_folder / 'data.gdb')


In [131]:
layer = data['layers'][0]
# feature_layer = FeatureLayer(layer['url'])
object_ids = ','.join([str(x) for x in layer['objectIds']])

path = f'ESRIJSON:{layer["url"]}/query?objectids={object_ids}&orderByFields=OBJECTID+ASC&outfields=*&f=json'
input_dataset = gdal.OpenEx(path)
input_layer = input_dataset.GetLayer()
feature = input_layer.GetNextFeature()
input_layer.GetName()


'ESRIJSON'

In [138]:
output_path = current_folder / 'data.gdb'
if output_path.exists():
  shutil.rmtree(output_path)

output_dataset = ogr.GetDriverByName('OpenFileGDB').CreateDataSource(current_folder / 'data.gdb')

output_layer = output_dataset.CreateLayer(layer['tableName'], srs=input_layer.GetSpatialRef(), geom_type=input_layer.GetGeomType())
for field in input_layer.schema:
  output_layer.CreateField(field)

for feature in input_layer:
  output_layer.CreateFeature(feature)

output_dataset.Close()


0

In [128]:
output_dataset = current_folder / 'data.gdb'
# if output_dataset.exists():
#   shutil.rmtree(output_dataset)
gdal.VectorTranslate(output_dataset, input_dataset,
  # forceNullable=True,
  skipFailures=True,
  format='OpenFileGDB',
  layerName=layer['tableName'],
)


RuntimeError: OpenFileGDB driver failed to create /workspaces/deq-enviro/functions-py/data.gdb
May be caused by: /workspaces/deq-enviro/functions-py/data.gdb already exists.